## Example script illustrating sound classification on audio stream
This notebook is an example how to use DeGirum PySDK to do sound classification AI inference of an audio stream from local microphone.

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you just need to uncomment **one** of the lines in the code below.

You also need to specify your cloud API access token, cloud zoo URLs, and AI server hostname in [env.ini](../../env.ini) file, located in the same directory as this notebook.

**pyaudio package with portaudio is required to run this sample.**

**Access to microphone is required to run this sample.**

In [ ]:
# make sure degirum-tools package is installed
!pip show degirum-tools || pip install degirum-tools

#### Specify where do you want to run your inferences

In [ ]:
import degirum as dg, degirum_tools

degirum_tools.configure_colab() # configure for Google Colab

#
# Please UNCOMMENT only ONE of the following lines to specify where to run AI inference
#

target = dg.CLOUD # <-- on the Cloud Platform
# target = degirum_tools.get_ai_server_hostname() # <-- on AI Server deployed in your LAN
# target = dg.LOCAL # <-- on ORCA accelerator installed on this computer

# connect to AI inference engine getting zoo URL and token from env.ini file
zoo = dg.connect(target, degirum_tools.get_cloud_zoo_url(), degirum_tools.get_token())

#### The rest of the cells below should run without any modifications

In [ ]:
import sys
import numpy as np
from IPython.display import clear_output

In [ ]:
# load YAMNET sound classification model for DeGirum Orca AI accelerator
# (change model name to "...n2x_cpu_1" to run it on CPU)
model = zoo.load_model("mobilenet_v1_yamnet_sound_cls--96x64_quant_n2x_orca1_1")

In [ ]:
abort = False  # stream abort flag
N = 5  # inference results history depth
history = []  # list of N consecutive inference results

sampling_rate_hz = model.model_info.InputSamplingRate[0]
read_buf_size = model.model_info.InputWaveformSize[0] // 2  # to have 50% overlap

# Acquire model input stream object
with degirum_tools.open_audio_stream(sampling_rate_hz, read_buf_size) as stream:
    #
    # AI prediction loop.
    # emit keyboard typing sound to stop
    #
    for res in model.predict_batch(
        degirum_tools.audio_overlapped_source(stream, lambda: abort)
    ):
        # add top inference result to history
        history.insert(0, f"{res.results[0]['label']}: {res.results[0]['score']}")
        if len(history) > N:  # keep only N last elements in history
            history.pop()

        clear_output(wait=True)  # clear Jupyter output cell
        for m in history:  # print history
            print(m)

        if res.results[0]["label"] == "Typing":  # check for stop condition
            abort = True